# Day17_1: RNN, LSTM, GRU (시퀀스 모델링)

## 학습 목표

**Part 1: 기초**
1. 시퀀스 데이터의 특성 이해하기
2. RNN(순환 신경망) 구조 이해하기
3. 기울기 소실 문제 이해하기
4. LSTM 게이트 메커니즘 이해하기
5. GRU 구조 이해하기

**Part 2: 심화**
1. 양방향 RNN (Bidirectional) 이해하기
2. 시계열 예측 실습하기 (주가 예측)
3. 텍스트 감성 분석 실습하기 (IMDB)
4. Plotly로 예측 결과 시각화하기

---

## 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 시퀀스 데이터 | 순서가 중요한 데이터 처리 | 주가, 텍스트, 센서 데이터 |
| RNN | 시간적 패턴 학습 | 음성 인식, 번역 |
| LSTM | 장기 의존성 학습 | 긴 문장 이해, 복잡한 시계열 |
| GRU | 효율적인 시퀀스 모델링 | 실시간 예측, 모바일 배포 |

**분석가 관점**: RNN 계열 모델은 시계열 예측과 자연어 처리의 핵심입니다. LSTM과 GRU는 긴 시퀀스에서도 중요한 정보를 기억할 수 있어, 주가 예측, 감성 분석, 번역 등 다양한 실무에서 활용됩니다!

---

# Part 1: 기초

---

## 1.1 시퀀스 데이터란?

### 시퀀스 데이터의 특징

**시퀀스 데이터**: 데이터 포인트들이 특정 순서로 배열되어 있고, 이 순서가 중요한 의미를 갖는 데이터

```
시퀀스 데이터 예시:
- 시계열: [100, 102, 105, 103, 108] (주가)
- 텍스트: ["오늘", "날씨가", "정말", "좋다"]
- 음성: [0.1, 0.2, 0.15, 0.3, ...] (파형)
- 센서: [22.1, 22.3, 22.5, 23.0] (온도)
```

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from torch.utils.data import Dataset, DataLoader

print(f"PyTorch 버전: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

In [ ]:
# 시퀀스 데이터 예시: 순서가 중요!
sentence1 = ["고양이가", "쥐를", "쫓는다"]
sentence2 = ["쥐가", "고양이를", "쫓는다"]

print("시퀀스 데이터에서 순서의 중요성:")
print(f"문장 1: {' '.join(sentence1)} (일반적인 상황)")
print(f"문장 2: {' '.join(sentence2)} (특이한 상황)")
print("\n같은 단어지만 순서가 다르면 의미가 완전히 달라집니다!")

### 기존 모델(DNN, CNN)의 한계

| 모델 | 문제점 |
|------|--------|
| DNN | 입력을 1차원 벡터로 취급 -> 순서 정보 파괴 |
| CNN | 공간적 특징 추출에 특화 -> 시간적 연속성 모델링 어려움 |

In [ ]:
# DNN의 문제: 순서 정보 손실
# "not good"과 "good not"은 DNN에게 같은 입력으로 보일 수 있음

words = {"good": [0.8, 0.1], "not": [-0.5, 0.3]}  # 간단한 임베딩

# DNN 스타일: 모든 단어를 concat해서 입력
input1 = np.concatenate([words["not"], words["good"]])  # "not good"
input2 = np.concatenate([words["good"], words["not"]])  # "good not"

print("DNN 입력 (순서 무시):")
print(f"'not good': {input1}")
print(f"'good not': {input2}")
print("\n순서가 바뀌어도 다르게 처리되지만, 순서의 '의미'를 모델이 배우기 어려움")

---

## 1.2 RNN (Recurrent Neural Network) 구조

### RNN의 핵심 아이디어: 순환 (Recurrence)

```
기존 DNN:  x -> [신경망] -> y (한 번에 처리)

RNN:       x1 -> [RNN] -> h1 (+ y1)
              ↓    ↑
           x2 -> [RNN] -> h2 (+ y2)  (이전 상태 활용)
              ↓    ↑
           x3 -> [RNN] -> h3 (+ y3)
```

### Hidden State (은닉 상태)

RNN의 '기억'을 담당하는 핵심 요소입니다.

```
h_t = tanh(W_hh * h_{t-1} + W_xh * x_t + b_h)

- h_t: 현재 시점의 은닉 상태
- h_{t-1}: 이전 시점의 은닉 상태
- x_t: 현재 시점의 입력
- W_hh, W_xh: 가중치 행렬 (모든 시점에서 공유!)
```

In [ ]:
# PyTorch nn.RNN 기본 사용법
batch_size = 2
seq_len = 5      # 시퀀스 길이 (예: 5개 단어)
input_size = 10  # 입력 차원 (예: 임베딩 차원)
hidden_size = 20 # 은닉 상태 차원

# RNN 레이어 정의
rnn = nn.RNN(
    input_size=input_size,
    hidden_size=hidden_size,
    num_layers=1,      # RNN 층 수
    batch_first=True   # 입력 shape: (batch, seq, feature)
)

# 더미 입력 데이터
x = torch.randn(batch_size, seq_len, input_size)
print(f"입력 shape: {x.shape}")
print(f"  - batch_size: {batch_size}")
print(f"  - seq_len: {seq_len}")
print(f"  - input_size: {input_size}")

In [ ]:
# RNN 순전파
# 초기 은닉 상태 (생략 가능, 기본값 0)
h0 = torch.zeros(1, batch_size, hidden_size)  # (num_layers, batch, hidden)

# 순전파
outputs, h_n = rnn(x, h0)

print(f"\nRNN 출력:")
print(f"  outputs shape: {outputs.shape}  # 모든 시점의 은닉 상태")
print(f"  h_n shape: {h_n.shape}          # 마지막 시점의 은닉 상태")
print(f"\noutputs[:, -1, :] == h_n: {torch.allclose(outputs[:, -1, :], h_n.squeeze(0))}")

### 실무 예시: RNN 출력 활용 방식

| 작업 | 사용하는 출력 | 예시 |
|------|-------------|------|
| 분류 (Many-to-One) | h_n (마지막 은닉) | 감성 분류 |
| 시퀀스 생성 (Many-to-Many) | outputs (모든 은닉) | 번역, 태깅 |
| 시계열 예측 | outputs[:, -1, :] | 다음 값 예측 |

In [ ]:
# 분류 모델 예시: 마지막 은닉 상태 -> 클래스 예측
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # x: (batch, seq_len, input_size)
        outputs, h_n = self.rnn(x)
        # h_n: (1, batch, hidden_size) -> (batch, hidden_size)
        out = self.fc(h_n.squeeze(0))
        return out

# 모델 생성
classifier = RNNClassifier(input_size=10, hidden_size=20, num_classes=3)
sample_input = torch.randn(4, 5, 10)  # batch=4, seq=5, input=10
output = classifier(sample_input)
print(f"분류 모델 출력 shape: {output.shape}  # (batch, num_classes)")

---

## 1.3 기울기 소실 문제 (Vanishing Gradient)

### 왜 발생하는가?

RNN에서 역전파 시 같은 가중치 행렬이 반복적으로 곱해집니다.

```
역전파 시 그래디언트:
dL/dW = dL/dh_T * dh_T/dh_{T-1} * ... * dh_2/dh_1 * dh_1/dW

문제:
- tanh의 미분값 범위: (0, 1]
- 긴 시퀀스에서 미분값이 계속 곱해짐 -> 0에 수렴
- 결과: 앞쪽 시점의 정보가 학습되지 않음
```

In [ ]:
# 기울기 소실 시뮬레이션
def simulate_gradient_flow(seq_len, gradient_multiplier=0.5):
    """시퀀스 길이에 따른 그래디언트 크기 시뮬레이션"""
    gradients = [1.0]  # 마지막 시점의 그래디언트 = 1
    for t in range(1, seq_len):
        # 역전파: 이전 그래디언트 * 가중치 미분
        gradients.append(gradients[-1] * gradient_multiplier)
    return gradients[::-1]  # 시간 순서로 뒤집기

# 다양한 시퀀스 길이
seq_lengths = [10, 30, 50, 100]
results = {}

for seq_len in seq_lengths:
    grads = simulate_gradient_flow(seq_len, gradient_multiplier=0.7)
    results[f"seq={seq_len}"] = grads
    print(f"시퀀스 길이 {seq_len:3d}: 첫 시점 그래디언트 = {grads[0]:.2e}")

In [ ]:
# 그래디언트 소실 시각화
fig = go.Figure()

for name, grads in results.items():
    fig.add_trace(go.Scatter(
        x=list(range(len(grads))),
        y=grads,
        mode='lines',
        name=name
    ))

fig.update_layout(
    title='기울기 소실 문제: 시퀀스 길이에 따른 그래디언트 크기',
    xaxis_title='시점 (Time Step)',
    yaxis_title='그래디언트 크기',
    yaxis_type='log',
    template='plotly_white'
)
fig.show()

### 장기 의존성 문제 (Long-term Dependency)

기울기 소실의 결과: 앞쪽의 중요한 정보가 뒤쪽으로 전달되지 않음

```
예시: "나는 프랑스에서 태어났고, ... (많은 문장들) ..., 그래서 [?]어를 잘한다."

- 정답: 프랑스어
- 문제: "프랑스"라는 정보가 긴 시퀀스를 지나면서 소실됨
```

---

## 1.4 LSTM (Long Short-Term Memory)

### LSTM의 핵심 아이디어

기울기 소실 문제를 해결하기 위해 **게이트 메커니즘**과 **셀 상태(Cell State)**를 도입

```
LSTM의 3가지 게이트:

1. 망각 게이트 (Forget Gate): 이전 기억 중 버릴 것 결정
   f_t = sigmoid(W_f * [h_{t-1}, x_t] + b_f)

2. 입력 게이트 (Input Gate): 새 정보 중 저장할 것 결정
   i_t = sigmoid(W_i * [h_{t-1}, x_t] + b_i)
   C_tilde = tanh(W_C * [h_{t-1}, x_t] + b_C)

3. 출력 게이트 (Output Gate): 셀 상태 중 출력할 것 결정
   o_t = sigmoid(W_o * [h_{t-1}, x_t] + b_o)
   h_t = o_t * tanh(C_t)

셀 상태 업데이트:
   C_t = f_t * C_{t-1} + i_t * C_tilde
```

In [ ]:
# LSTM 게이트 시각화를 위한 예시
def visualize_gates():
    """LSTM 게이트 동작 시뮬레이션"""
    # 예시 시나리오: "나는 고양이를 좋아한다. 강아지도 좋다."
    time_steps = ['나는', '고양이를', '좋아한다', '.', '강아지도', '좋다', '.']
    
    # 게이트 값 시뮬레이션 (실제로는 학습됨)
    forget_gate = [0.9, 0.8, 0.7, 0.3, 0.9, 0.8, 0.3]  # 마침표에서 많이 잊음
    input_gate = [0.9, 0.9, 0.8, 0.1, 0.9, 0.8, 0.1]   # 마침표에서 적게 저장
    output_gate = [0.5, 0.8, 0.9, 0.2, 0.8, 0.9, 0.2]  # 동사에서 많이 출력
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(name='Forget Gate', x=time_steps, y=forget_gate, marker_color='red'))
    fig.add_trace(go.Bar(name='Input Gate', x=time_steps, y=input_gate, marker_color='green'))
    fig.add_trace(go.Bar(name='Output Gate', x=time_steps, y=output_gate, marker_color='blue'))
    
    fig.update_layout(
        title='LSTM 게이트 동작 시뮬레이션',
        xaxis_title='단어 (시점)',
        yaxis_title='게이트 값 (0~1)',
        barmode='group',
        template='plotly_white'
    )
    return fig

fig = visualize_gates()
fig.show()

In [ ]:
# PyTorch nn.LSTM 사용법
lstm = nn.LSTM(
    input_size=10,
    hidden_size=20,
    num_layers=1,
    batch_first=True
)

# 입력
x = torch.randn(2, 5, 10)  # batch=2, seq=5, input=10

# 초기 상태: (h_0, c_0) - hidden state와 cell state
h0 = torch.zeros(1, 2, 20)  # (num_layers, batch, hidden)
c0 = torch.zeros(1, 2, 20)  # cell state

# 순전파
outputs, (h_n, c_n) = lstm(x, (h0, c0))

print("LSTM 출력:")
print(f"  outputs: {outputs.shape}  # 모든 시점의 hidden state")
print(f"  h_n: {h_n.shape}          # 마지막 hidden state")
print(f"  c_n: {c_n.shape}          # 마지막 cell state (LSTM 특유)")

### 실무 예시: LSTM으로 긴 시퀀스 학습

LSTM은 셀 상태(Cell State)라는 '고속도로'를 통해 정보가 손실 없이 흐를 수 있습니다.

```
RNN:  정보 흐름이 매 단계 tanh를 거침 -> 소실
LSTM: 셀 상태는 덧셈/곱셈만 -> 정보 보존
```

---

## 1.5 GRU (Gated Recurrent Unit)

### GRU: LSTM의 간소화 버전

LSTM의 3개 게이트를 2개로 줄이고, 셀 상태 없이 은닉 상태만 사용

```
GRU의 2가지 게이트:

1. 리셋 게이트 (Reset Gate): 이전 정보를 얼마나 무시할지
   r_t = sigmoid(W_r * [h_{t-1}, x_t] + b_r)

2. 업데이트 게이트 (Update Gate): 새 정보와 이전 정보의 비율
   z_t = sigmoid(W_z * [h_{t-1}, x_t] + b_z)

은닉 상태 업데이트:
   h_tilde = tanh(W_h * [r_t * h_{t-1}, x_t] + b_h)
   h_t = (1 - z_t) * h_{t-1} + z_t * h_tilde
```

In [ ]:
# LSTM vs GRU 비교표
comparison = {
    "구분": ["게이트 수", "상태", "파라미터 수", "학습 속도", "성능", "적합한 상황"],
    "LSTM": [
        "3개 (Forget, Input, Output)",
        "Hidden State + Cell State",
        "많음 (4 * hidden^2)",
        "느림",
        "긴 시퀀스에서 약간 우위",
        "복잡한 장기 의존성"
    ],
    "GRU": [
        "2개 (Reset, Update)",
        "Hidden State만",
        "적음 (3 * hidden^2)",
        "빠름",
        "대부분 LSTM과 비슷",
        "적은 데이터, 빠른 학습"
    ]
}

comparison_df = pd.DataFrame(comparison)
print("LSTM vs GRU 비교")
print("="*60)
print(comparison_df.to_string(index=False))

In [ ]:
# PyTorch nn.GRU 사용법
gru = nn.GRU(
    input_size=10,
    hidden_size=20,
    num_layers=1,
    batch_first=True
)

# 입력
x = torch.randn(2, 5, 10)

# 순전파 (GRU는 hidden state만!)
outputs, h_n = gru(x)

print("GRU 출력:")
print(f"  outputs: {outputs.shape}")
print(f"  h_n: {h_n.shape}  # cell state 없음!")

In [ ]:
# 파라미터 수 비교
def count_params(model):
    return sum(p.numel() for p in model.parameters())

input_size, hidden_size = 100, 256

rnn = nn.RNN(input_size, hidden_size, batch_first=True)
lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
gru = nn.GRU(input_size, hidden_size, batch_first=True)

print(f"파라미터 수 비교 (input={input_size}, hidden={hidden_size}):")
print(f"  RNN:  {count_params(rnn):,}")
print(f"  LSTM: {count_params(lstm):,} (RNN의 약 4배)")
print(f"  GRU:  {count_params(gru):,}  (RNN의 약 3배)")

---

# Part 2: 심화

---

## 2.1 양방향 RNN (Bidirectional RNN)

### 왜 양방향인가?

일반 RNN은 **과거 -> 현재** 방향으로만 정보가 흐릅니다.
하지만 텍스트에서는 **미래 정보**도 중요할 수 있습니다.

```
예시: "나는 ___를 먹었다. 정말 달콤했다."
- 빈칸을 채우려면 '달콤했다'라는 미래 정보가 필요
- 양방향 RNN: 과거 + 미래 정보 모두 활용
```

In [ ]:
# 양방향 LSTM
bi_lstm = nn.LSTM(
    input_size=10,
    hidden_size=20,
    num_layers=1,
    batch_first=True,
    bidirectional=True  # 양방향!
)

x = torch.randn(2, 5, 10)
outputs, (h_n, c_n) = bi_lstm(x)

print("양방향 LSTM 출력:")
print(f"  outputs: {outputs.shape}  # hidden_size * 2 = 40")
print(f"  h_n: {h_n.shape}          # (num_layers * 2, batch, hidden)")
print(f"\n양방향이므로 출력 차원이 2배!")

In [ ]:
# 양방향 LSTM 분류 모델
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size,
            batch_first=True,
            bidirectional=True
        )
        # 양방향이므로 hidden_size * 2
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x):
        outputs, (h_n, c_n) = self.lstm(x)
        # 순방향 마지막 + 역방향 마지막 concat
        # h_n: (2, batch, hidden)
        forward_h = h_n[0]   # 순방향
        backward_h = h_n[1]  # 역방향
        hidden = torch.cat([forward_h, backward_h], dim=1)
        return self.fc(hidden)

model = BiLSTMClassifier(10, 20, 3)
output = model(torch.randn(4, 5, 10))
print(f"양방향 LSTM 분류 출력: {output.shape}")

---

## 2.2 시계열 예측 실습 (주가 예측)

### 문제 정의

과거 N일의 주가로 다음 날 주가를 예측하는 **Many-to-One** 모델

```
입력: [Day1, Day2, ..., Day30] (30일 주가)
출력: [Day31] (다음 날 예측)
```

In [ ]:
# 주가 데이터 생성 (시뮬레이션)
np.random.seed(42)

# 트렌드 + 계절성 + 노이즈
days = 500
trend = np.linspace(100, 150, days)
seasonal = 10 * np.sin(np.linspace(0, 8*np.pi, days))
noise = np.random.randn(days) * 3
stock_price = trend + seasonal + noise

# 시각화
fig = px.line(
    x=range(days), y=stock_price,
    title='시뮬레이션 주가 데이터',
    labels={'x': '날짜', 'y': '주가'}
)
fig.update_layout(template='plotly_white')
fig.show()

In [ ]:
# 데이터 전처리: 정규화 + 시퀀스 생성
from sklearn.preprocessing import MinMaxScaler

# 정규화 (0~1)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(stock_price.reshape(-1, 1))

# 시퀀스 데이터 생성
def create_sequences(data, seq_length):
    """과거 seq_length일로 다음 날 예측하는 데이터셋 생성"""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

SEQ_LENGTH = 30
X, y = create_sequences(scaled_data, SEQ_LENGTH)

print(f"시퀀스 데이터:")
print(f"  X shape: {X.shape}  # (샘플 수, 시퀀스 길이, 특성)")
print(f"  y shape: {y.shape}  # (샘플 수, 특성)")

In [ ]:
# 훈련/테스트 분할
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 텐서 변환
X_train_t = torch.FloatTensor(X_train)
y_train_t = torch.FloatTensor(y_train)
X_test_t = torch.FloatTensor(X_test)
y_test_t = torch.FloatTensor(y_test)

print(f"훈련 데이터: {X_train_t.shape}")
print(f"테스트 데이터: {X_test_t.shape}")

# DataLoader
train_dataset = torch.utils.data.TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# LSTM 시계열 예측 모델
class StockPredictor(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2
        )
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        # x: (batch, seq_len, input_size)
        lstm_out, (h_n, c_n) = self.lstm(x)
        # 마지막 시점의 출력 사용
        out = self.fc(lstm_out[:, -1, :])
        return out

# 모델 생성
model = StockPredictor(input_size=1, hidden_size=50, num_layers=2).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

In [ ]:
# 학습
epochs = 50
train_losses = []

X_train_t = X_train_t.to(device)
y_train_t = y_train_t.to(device)
X_test_t = X_test_t.to(device)
y_test_t = y_test_t.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")

In [ ]:
# 예측 및 역정규화
model.eval()
with torch.no_grad():
    train_pred = model(X_train_t).cpu().numpy()
    test_pred = model(X_test_t).cpu().numpy()

# 역정규화
train_pred_inv = scaler.inverse_transform(train_pred)
test_pred_inv = scaler.inverse_transform(test_pred)
y_train_inv = scaler.inverse_transform(y_train_t.cpu().numpy())
y_test_inv = scaler.inverse_transform(y_test_t.cpu().numpy())

# 평가 지표
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test_inv, test_pred_inv)
rmse = np.sqrt(mean_squared_error(y_test_inv, test_pred_inv))

print(f"\n테스트 성능:")
print(f"  MAE: {mae:.2f}")
print(f"  RMSE: {rmse:.2f}")

In [ ]:
# 예측 결과 시각화
fig = go.Figure()

# 전체 실제 데이터
fig.add_trace(go.Scatter(
    x=list(range(len(stock_price))),
    y=stock_price,
    mode='lines',
    name='실제 주가',
    line=dict(color='blue')
))

# 테스트 예측
test_idx = list(range(train_size + SEQ_LENGTH, len(stock_price)))
fig.add_trace(go.Scatter(
    x=test_idx,
    y=test_pred_inv.flatten(),
    mode='lines',
    name='LSTM 예측',
    line=dict(color='red', dash='dash')
))

# 훈련/테스트 분리선
fig.add_vline(x=train_size + SEQ_LENGTH, line_dash='dot', line_color='green',
              annotation_text='Train/Test 분리')

fig.update_layout(
    title=f'LSTM 주가 예측 결과 (MAE: {mae:.2f}, RMSE: {rmse:.2f})',
    xaxis_title='날짜',
    yaxis_title='주가',
    template='plotly_white'
)
fig.show()

---

## 2.3 텍스트 감성 분석 실습 (IMDB)

### 문제 정의

영화 리뷰 텍스트를 보고 긍정(1) / 부정(0) 분류

```
입력: "This movie was great!" -> [토큰1, 토큰2, 토큰3, 토큰4]
출력: 1 (긍정)
```

In [ ]:
# 간단한 감성 분류 데이터셋 (IMDB 대신 간단한 예시)
# 실제로는 torchtext나 datasets 라이브러리 사용

# 간단한 단어 사전
vocab = {
    '<PAD>': 0, '<UNK>': 1,
    'this': 2, 'movie': 3, 'was': 4, 'is': 5,
    'great': 6, 'good': 7, 'bad': 8, 'terrible': 9,
    'amazing': 10, 'awful': 11, 'boring': 12, 'exciting': 13,
    'the': 14, 'a': 15, 'very': 16, 'really': 17,
    'loved': 18, 'hated': 19, 'it': 20, 'film': 21
}
vocab_size = len(vocab)

# 샘플 데이터 생성
positive_reviews = [
    "this movie was great",
    "the film is amazing",
    "really good movie",
    "loved this film",
    "very exciting movie"
]

negative_reviews = [
    "this movie was bad",
    "the film is terrible",
    "really awful movie",
    "hated this film",
    "very boring movie"
]

print(f"어휘 크기: {vocab_size}")
print(f"긍정 리뷰 예시: {positive_reviews[0]}")
print(f"부정 리뷰 예시: {negative_reviews[0]}")

In [ ]:
# 텍스트 -> 토큰 인덱스 변환
def text_to_indices(text, vocab, max_len=10):
    tokens = text.lower().split()
    indices = [vocab.get(t, vocab['<UNK>']) for t in tokens]
    # 패딩 또는 자르기
    if len(indices) < max_len:
        indices = indices + [vocab['<PAD>']] * (max_len - len(indices))
    else:
        indices = indices[:max_len]
    return indices

# 데이터셋 준비
max_len = 10
X_data = []
y_data = []

for review in positive_reviews:
    X_data.append(text_to_indices(review, vocab, max_len))
    y_data.append(1)

for review in negative_reviews:
    X_data.append(text_to_indices(review, vocab, max_len))
    y_data.append(0)

# 데이터 증강 (간단히 복제)
X_data = X_data * 20  # 100개로 증강
y_data = y_data * 20

X_tensor = torch.LongTensor(X_data)
y_tensor = torch.FloatTensor(y_data).reshape(-1, 1)

print(f"데이터 shape: X={X_tensor.shape}, y={y_tensor.shape}")
print(f"토큰 예시: {X_tensor[0].tolist()}")

In [ ]:
# LSTM 감성 분류 모델
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers=1):
        super().__init__()
        # 임베딩 레이어: 단어 인덱스 -> 벡터
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        
        # LSTM
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )
        
        # 분류 레이어
        self.fc = nn.Linear(hidden_size * 2, 1)  # 양방향
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # x: (batch, seq_len) - 토큰 인덱스
        embedded = self.embedding(x)  # (batch, seq_len, embed_dim)
        
        lstm_out, (h_n, c_n) = self.lstm(embedded)
        
        # 양방향 마지막 hidden 결합
        hidden = torch.cat([h_n[-2], h_n[-1]], dim=1)
        
        out = self.fc(hidden)
        return self.sigmoid(out)

# 모델 생성
sentiment_model = SentimentLSTM(
    vocab_size=vocab_size,
    embed_dim=32,
    hidden_size=64,
    num_layers=1
).to(device)

print(sentiment_model)

In [ ]:
# 학습
criterion = nn.BCELoss()
optimizer = optim.Adam(sentiment_model.parameters(), lr=0.01)

# DataLoader
dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

epochs = 50
losses = []

for epoch in range(epochs):
    sentiment_model.train()
    total_loss = 0
    
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        output = sentiment_model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    losses.append(total_loss / len(loader))
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {losses[-1]:.4f}")

In [ ]:
# 예측 테스트
def predict_sentiment(text, model, vocab):
    model.eval()
    indices = text_to_indices(text, vocab)
    x = torch.LongTensor([indices]).to(device)
    with torch.no_grad():
        prob = model(x).item()
    sentiment = "긍정" if prob > 0.5 else "부정"
    return sentiment, prob

# 테스트
test_reviews = [
    "this movie was great",
    "the film is terrible",
    "really loved it",
    "very bad film"
]

print("감성 분석 결과:")
print("="*50)
for review in test_reviews:
    sentiment, prob = predict_sentiment(review, sentiment_model, vocab)
    print(f"'{review}'")
    print(f"  -> {sentiment} (확률: {prob:.3f})")

---

## 2.4 Plotly로 예측 결과 시각화

### 학습 곡선 및 모델 비교

In [ ]:
# 주가 예측 학습 곡선
fig = make_subplots(rows=1, cols=2, subplot_titles=['주가 예측 LSTM', '감성 분류 LSTM'])

# 주가 예측 손실
fig.add_trace(
    go.Scatter(y=train_losses, mode='lines', name='주가 예측 Loss'),
    row=1, col=1
)

# 감성 분류 손실
fig.add_trace(
    go.Scatter(y=losses, mode='lines', name='감성 분류 Loss'),
    row=1, col=2
)

fig.update_xaxes(title_text='Epoch')
fig.update_yaxes(title_text='Loss')
fig.update_layout(title='LSTM 학습 곡선', height=400, template='plotly_white')
fig.show()

In [ ]:
# RNN vs LSTM vs GRU 비교 (간단한 시퀀스 문제)
def train_and_compare(model_type, X_train, y_train, epochs=30):
    """다양한 시퀀스 모델 훈련 및 비교"""
    torch.manual_seed(42)
    
    if model_type == 'RNN':
        rnn_layer = nn.RNN(1, 32, batch_first=True)
    elif model_type == 'LSTM':
        rnn_layer = nn.LSTM(1, 32, batch_first=True)
    else:  # GRU
        rnn_layer = nn.GRU(1, 32, batch_first=True)
    
    model = nn.Sequential(
        rnn_layer,
    )
    
    fc = nn.Linear(32, 1)
    optimizer = optim.Adam(list(model.parameters()) + list(fc.parameters()), lr=0.01)
    criterion = nn.MSELoss()
    
    losses = []
    for epoch in range(epochs):
        if model_type == 'LSTM':
            out, (h, c) = model(X_train)
        else:
            out, h = model(X_train)
        
        pred = fc(out[:, -1, :])
        loss = criterion(pred, y_train)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
    
    return losses

# 비교
rnn_losses = train_and_compare('RNN', X_train_t.cpu()[:100], y_train_t.cpu()[:100])
lstm_losses = train_and_compare('LSTM', X_train_t.cpu()[:100], y_train_t.cpu()[:100])
gru_losses = train_and_compare('GRU', X_train_t.cpu()[:100], y_train_t.cpu()[:100])

# 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(y=rnn_losses, mode='lines', name='RNN'))
fig.add_trace(go.Scatter(y=lstm_losses, mode='lines', name='LSTM'))
fig.add_trace(go.Scatter(y=gru_losses, mode='lines', name='GRU'))

fig.update_layout(
    title='RNN vs LSTM vs GRU 학습 곡선 비교',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    template='plotly_white'
)
fig.show()

---

## 실습 퀴즈

**난이도**: (쉬움) ~ (어려움)

---

### Q1. RNN 출력 shape 계산하기

**문제**: 다음 RNN의 출력 shape을 계산하세요.

```python
rnn = nn.RNN(input_size=50, hidden_size=100, num_layers=2, batch_first=True)
x = torch.randn(16, 20, 50)  # batch=16, seq=20, input=50
outputs, h_n = rnn(x)
```

outputs와 h_n의 shape은?

In [ ]:
import torch
import torch.nn as nn

# 여기에 코드를 작성하세요


### Q2. Hidden State 이해하기

**문제**: RNN의 마지막 시점 출력(outputs[:, -1, :])과 h_n이 같은지 확인하세요.

In [ ]:
import torch
import torch.nn as nn

# 여기에 코드를 작성하세요


### Q3. 시퀀스 데이터 특성

**문제**: 다음 중 시퀀스 데이터가 아닌 것을 고르고 이유를 설명하세요.

1. 일별 주가
2. 영화 리뷰 텍스트
3. 학생들의 키와 몸무게
4. 음성 파형

In [ ]:
# 여기에 답과 이유를 작성하세요


### Q4. LSTM 게이트 이해하기

**문제**: LSTM의 3가지 게이트(Forget, Input, Output)의 역할을 각각 한 문장으로 설명하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q5. GRU와 LSTM 차이

**문제**: GRU가 LSTM보다 파라미터 수가 적은 이유를 설명하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q6. 양방향 RNN

**문제**: 양방향 LSTM을 정의하고, 출력 shape을 확인하세요.

- input_size=30, hidden_size=64, num_layers=1
- 입력: batch=8, seq=15, input=30

In [ ]:
import torch
import torch.nn as nn

# 여기에 코드를 작성하세요


### Q7. 시계열 데이터 전처리

**문제**: 아래 주가 데이터로 시퀀스 길이 5인 학습 데이터를 생성하세요.

```python
prices = [100, 102, 105, 103, 108, 110, 107, 112, 115, 113]
```

힌트: 5일 데이터로 다음 날 예측

In [ ]:
import numpy as np

prices = [100, 102, 105, 103, 108, 110, 107, 112, 115, 113]

# 여기에 코드를 작성하세요


### Q8. 임베딩 레이어 이해

**문제**: 어휘 크기 1000, 임베딩 차원 64인 Embedding 레이어를 생성하고, 배치 크기 4, 시퀀스 길이 10인 입력의 출력 shape을 확인하세요.

In [ ]:
import torch
import torch.nn as nn

# 여기에 코드를 작성하세요


### Q9. 주가 예측 LSTM 구현

**문제**: 아래 사인파 데이터로 다음 값을 예측하는 LSTM 모델을 구현하고 학습하세요.

요구사항:
1. 시퀀스 길이: 20
2. LSTM hidden_size: 32
3. 30 에포크 학습
4. 예측 결과 시각화

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# 사인파 데이터
x = np.linspace(0, 20*np.pi, 500)
y = np.sin(x)

# 여기에 코드를 작성하세요


### Q10. 감성 분석 모델 개선

**문제**: 본문의 SentimentLSTM 모델을 개선하세요.

요구사항:
1. GRU로 변경
2. Dropout 추가 (0.3)
3. 2층 구조로 변경
4. 학습 후 테스트 리뷰에 대한 예측 출력

In [ ]:
import torch
import torch.nn as nn

# 여기에 코드를 작성하세요


---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|-----|----------|----------|
| 시퀀스 데이터 | 순서가 중요한 데이터 | 시계열, 텍스트, 음성 |
| RNN | h_t = f(h_{t-1}, x_t) | 순차 패턴 학습 |
| 기울기 소실 | 긴 시퀀스에서 학습 어려움 | LSTM/GRU로 해결 |
| LSTM | 3개 게이트 + Cell State | 장기 의존성 학습 |
| GRU | 2개 게이트 (간소화) | 빠른 학습, 적은 데이터 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 내용 | 언제 사용? |
|-----|----------|----------|
| 양방향 RNN | 과거 + 미래 정보 활용 | 텍스트 분류, NER |
| 시계열 예측 | Many-to-One | 주가, 날씨 예측 |
| 감성 분석 | Embedding + LSTM | 리뷰 분류 |
| 시각화 | Plotly | 예측 vs 실제 비교 |

### RNN 계열 모델 선택 가이드

```
1. 기본 시작: GRU (빠르고 간단)
2. 성능 부족 시: LSTM으로 전환
3. 텍스트 분류: Bidirectional 추가
4. 시계열 예측: 단방향으로 충분
```

### 실무 팁

1. **데이터 정규화**: 시계열 데이터는 반드시 정규화 (MinMaxScaler 또는 StandardScaler)
2. **시퀀스 길이**: 너무 길면 학습 어려움, 도메인에 맞게 설정
3. **Dropout**: 과적합 방지, 0.2~0.5 사이
4. **Gradient Clipping**: 기울기 폭발 방지 (torch.nn.utils.clip_grad_norm_)
5. **Early Stopping**: 검증 손실 기반 조기 종료